In [16]:
import pandas as pd
import os, glob, re

In [30]:
unit_test_regex = re.compile('Testing \[(\w*)\]\s*(Ok|Failed)')

def parse_unit_log(filename):
    results = {}
    with open(filename) as fh:
        for line in fh:
            match = unit_test_regex.search(line)
            if match:
                results[match.group(1)] = match.group(2)
    return results

In [49]:
app_test_regex = re.compile('Testing \[(\w*)\]')
app_test_case_regex = re.compile('(OK|Failed) ... Case \[(\w*)\]')

def parse_app_log(filename):
    results = {}
    with open(filename) as fh:
        for line in fh:
            app_match = app_test_regex.search(line)
            if app_match:
                current_app = app_match.group(1)
                results[current_app] = {}
            case_match = app_test_case_regex.search(line)
            if case_match:
                results[current_app][case_match.group(2)] = case_match.group(1)
    return results

In [51]:
cat_package_regex = re.compile('Working on Package \[(\w*)\]')
cat_test_regex = re.compile('(OK|Failed) ... Case \[(\w*)\]')

def parse_cat_log(filename):
    results = {}
    with open(filename) as fh:
        for line in fh:
            package_match = cat_package_regex.search(line)
            if package_match:
                current_package = package_match.group(1)
                results[current_package] = {}
            test_match = cat_test_regex.search(line)
            if test_match:
                results[current_package][test_match.group(2)] = test_match.group(1)
    return results

In [6]:
logs_dir = "/usgs/cpkgs/isis3/logs/builds-tests/"
CentOS7 = "Linux_x86_64_prog24"
Mac10_11 = "Darwin_i386_prog26"
app_test_glob = "appTest*.log"
cat_test_glob = "catTest*.log"
unit_test_glob = "unitTest*.log"

In [15]:
centos_log_dir = os.path.join(logs_dir, CentOS7)
centos_app_logs = glob.glob(os.path.join(centos_log_dir, app_test_glob))
centos_cat_logs = glob.glob(os.path.join(centos_log_dir, cat_test_glob))
centos_unit_logs = glob.glob(os.path.join(centos_log_dir, unit_test_glob))

mac_log_dir = os.path.join(logs_dir, Mac10_11)
mac_app_logs = glob.glob(os.path.join(mac_log_dir, app_test_glob))
mac_cat_logs = glob.glob(os.path.join(mac_log_dir, cat_test_glob))
mac_unit_logs = glob.glob(os.path.join(mac_log_dir, unit_test_glob))